# Model Based RecSys

As before, the first thing is to load the data and get the most popular 50 movies, together with users that rated at least 6 of those movies. 

In [1]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.linalg import solve
from collections import defaultdict
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader 

## Load MovieLens Data

In [2]:
data_dir = "../data/ml-20m"
ratings = pd.read_csv(f"{data_dir}/ratings.csv")

In [3]:
def get_records_for(ratings, n_most_popular_movies=50, user_min_seen_movies=5):
    """
    ratings: pandas dataframe with raitngs
    n_most_popular_movies: Number of desired most common movies to get info from.
    user_min_seen_movies: Least number of movies that each user has rated.
    returns: pandas dataframe containing only information for the top most_n movies
    """
    # TO DO: Write your code here
    pass

Here is a sample solution for `get_records_for`:

In [4]:
def get_records_for(ratings, n_most_popular_movies=50, user_min_seen_movies=5):
    """
    ratings: pandas dataframe with raitngs
    n_most_popular_movies: Number of desired most common movies to get info from.
    user_min_seen_movies: Least number of movies that each user has rated.
    returns: pandas dataframe containing only information for the top most_n movies
    """
    
    ratings = ratings.drop("timestamp", axis=1)
    
    popular = ratings[['movieId', 'userId']].groupby('movieId', as_index=False)\
        .agg(len).sort_values('userId').tail(n_most_popular_movies)
    
    ratings = ratings[ratings['movieId'].isin(popular['movieId'].values)].copy(deep=True)
    
    lens = ratings.groupby("userId", as_index=False).agg(len)
    relevant_users = lens[lens['movieId'] >= user_min_seen_movies].userId
    ratings = ratings[ratings['userId'].isin(relevant_users)]
    
    return ratings 

ratings = get_records_for(ratings)

Note that the resulting information correspond to a user x item matrix

In [5]:
unique = ratings.nunique()
print("There are {} unique users".format(unique["userId"]))
print("Only {:.2f}% of the possible ratings are known".format(ratings.shape[0]/(unique["userId"] * 50)))

There are 110676 unique users
Only 0.38% of the possible ratings are known


We should create some testing dataset.

In [6]:
# Exercise: Make this faster?
def create_matrix(df):

    idx2user = df.userId.unique()
    user2idx = {user:i for i, user in enumerate(idx2user)}
    
    idx2movie = df.movieId.unique()
    movie2idx = {movie:i for i, movie in enumerate(idx2movie)}
    
    train = np.zeros((len(user2idx), len(movie2idx)))
    
    for _, row in df.iterrows():
        
        train[user2idx[row['userId']], movie2idx[row['movieId']]] = row['rating']
    
    test = np.zeros((len(user2idx), len(movie2idx)))
    
    for user_train, user_test in zip(train, test):
        
        nonzero = user_train.nonzero()[0]
        
        # Select 20% for testing
        size = nonzero.shape[0]//5
        test_ratings = np.random.choice(nonzero, size=size, replace=False)
        
        # Keep the records for testing
        user_test[test_ratings] = user_train[test_ratings]
        # Zero out for training
        user_train[test_ratings] = 0
        
    return train, test, user2idx, movie2idx

train, test, user2idx, movie2idx = create_matrix(ratings)

# Matrix Factorization

The next step is to do matrix factorization. To do this we recall that the goal takes the form 

$$L_{exp} = \sum\limits_{u,i \in S}(r_{ui} - \textbf{x}_{u}^{\intercal} \cdot{} \textbf{y}_{i})^{2} + \lambda_{x} \sum\limits_{u} \left\Vert \textbf{x}_{u} \right\Vert^{2} + \lambda_{y} \sum\limits_{u} \left\Vert \textbf{y}_{i} \right\Vert^{2}$$

or more advanced,

$$L_{WRMF} = \sum\limits_{u,i}c_{ui} \big( p_{ui} - \textbf{x}_{u}^{\intercal} \cdot{} \textbf{y}_{i} \big) ^{2} + \lambda_{x} \sum\limits_{u} \left\Vert \textbf{x}_{u} \right\Vert^{2} + \lambda_{y} \sum\limits_{u} \left\Vert \textbf{y}_{i} \right\Vert^{2}$$

We only focussed on the first type and leave the second one as an exercise.

We create a class that will take care of the model, a class should be able to do train and inference. Recall that the update rule is given by:

$$ x_u = (M M^T + \lambda_x \mathbb{Id})^{-1} M R$$

**Exercise**: Understand the difference between this and the Netflix paper update rule.

In [7]:
class MF:
    
    def __init__(self, user_x_item, rank, lambda_usr=0, lambda_item=0):
        
        self.data = user_x_item
        self.nb_users, self.nb_items = self.data.shape
        self.rank = rank
        self.users = np.random.random((self.nb_users, self.rank))
        self.items = np.random.random((self.nb_items, self.rank))
        self.lambda_usr = lambda_usr
        self.lambda_item = lambda_item
    
        
    def train(self, nb_steps, test=None):
        
        for i in range(nb_steps):
            
            self.update_user()
            self.update_item()
            
            if test is not None:
                print("Step: {}, error: {:.4f}, test_error: {:.4f}".\
                      format(i, self.error(), self.valid_error(test)), end="\r")
            else:
                print("Step: {}, error: {:.4f}".format(i, self.error()), end="\r")

        print("\n")
            
    def update_user(self):
        
        # Exercise: Complete this part
        YTY = (self.items.T).dot(self.items)
        reg = self.lambda_usr * np.eye(self.rank)
        
        for user_idx in range(self.nb_users):
            self.users[user_idx, :] = \
                solve(YTY + reg, (self.data[user_idx, :].T).dot(self.items))
        
    
    def update_item(self):
        
        # Exercise: Complete this part
        XTX = (self.users.T).dot(self.users)
        reg = self.lambda_item * np.eye(self.rank)
        
        for item_idx in range(self.nb_items):
            self.items[item_idx, :] = \
                solve(XTX + reg, (self.data[:, item_idx].T).dot(self.users))
    
    def error(self):
        
        return np.sqrt((((self.data - np.matmul(self.users, self.items.T)) ** 2).mean()))
    
    def valid_error(self, test):
        
        preds = np.matmul(self.users, self.items.T)
        
        error = 0
        counter = 0
        
        for user, pred in zip(test, preds):
            
            nonzero = user.nonzero()[0]
            counter += nonzero.shape[0]
            error += ((user[nonzero] - pred[nonzero])**2).sum()
        
        return np.sqrt(error/counter)
            
    
    
mf = MF(train, 20, 0.4, 0.4)
mf.train(10, test)

Step: 9, error: 1.0809, test_error: 2.9336



We can see convergence occurs rapidly. 

# A back propagation approach

The question that we have is a minimization question, so we could use back-propagation technique to solve it, we do this next

In [8]:
class MFD(nn.Module):
    
    def __init__(self, n_users, n_items, rank):
        
        super(MFD, self).__init__()
        
        self.users = nn.Embedding(n_users, rank)
        self.items = nn.Embedding(n_items, rank)
        self.rank = rank
    
    
    def forward(self, users, items):
        
        users = self.users(users)
        items = self.items(items)
        
        users = users.unsqueeze(1)
        items = items.unsqueeze(2)

        ranks = torch.bmm(users, items)
        
        return ranks.squeeze()
    

And we create the training:

In [9]:
def train(epochs, ratings, user2idx, movie2idx):
    
    nb_users = ratings.userId.nunique()
    nb_items = ratings.movieId.nunique()
    train_ratings, test_ratings = train_test_split(ratings, test_size=0.2)
    
    users = torch.tensor([user2idx[u] for u in train_ratings.userId]).long()
    movies = torch.tensor([movie2idx[m] for m in train_ratings.movieId]).long()
    ratings = torch.tensor(train_ratings.rating.values).float()
    
    data = TensorDataset(users, movies, ratings)
    dataloader = DataLoader(data, batch_size=4096, shuffle=True)
    
    users_valid = torch.tensor([user2idx[u] for u in test_ratings.userId]).long()
    movies_valid = torch.tensor([movie2idx[m] for m in test_ratings.movieId]).long()
    ratings_valid = torch.tensor(test_ratings.rating.values).float()
      
    model = MFD(nb_users, nb_items, 30)
    
    optim = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=1)
    
    for i in range(epochs):
        for users, items, ratings in dataloader:
            
            optim.zero_grad()
            pred_rat = model(users, items)
            loss = ((pred_rat - ratings)**2).mean()
            loss.backward()
            optim.step()
            print("Epoch {}/{} loss {}".format(i+1, epochs, torch.sqrt(loss)), end="\r")
        
        model.eval()
        pred_rat = model(users_valid, movies_valid)
        loss = ((pred_rat - ratings_valid)**2).mean()   
        print("Epoch {}/{} valid_loss {}".format(i+1, epochs, torch.sqrt(loss)))
        model.train()
        
train(10, ratings, user2idx, movie2idx)

Epoch 1/10 valid_loss 4.636169910430908
Epoch 2/10 valid_loss 4.132117748260498
Epoch 3/10 valid_loss 4.0301432609558105
Epoch 4/10 valid_loss 4.010544300079346
Epoch 5/10 valid_loss 4.006823539733887
Epoch 6/10 valid_loss 4.006123065948486
Epoch 7/10 valid_loss 4.005992889404297
Epoch 8/10 valid_loss 4.005970001220703
Epoch 9/10 valid_loss 4.0059661865234375
Epoch 10/10 valid_loss 4.005965709686279


In [10]:
torch.bmm(torch.rand(100,1, 3), torch.rand(100, 3, 1)).squeeze().size()

torch.Size([100])